In [17]:
!pip install z3-solver
from z3 import *

In [24]:
# Variables proposicionales para los alumnos y sus salas correspondientes

sala1 = [Bool(f"x_{i}_1") for i in range(10)]
sala2 = [Bool(f"x_{i}_2") for i in range(10)]
sala3 = [Bool(f"x_{i}_3") for i in range(10)]

# Variables proposicionales para las parejas que deben ir en distintas salas

parejas = [
    [Bool(f"p_{i}_{j}") if i < j else None for j in range(10)]
    for i in range(10)
]

print(sala1)
print(sala2)
print(sala3)
print(parejas)

[x_0_1, x_1_1, x_2_1, x_3_1, x_4_1, x_5_1, x_6_1, x_7_1, x_8_1, x_9_1]
[x_0_2, x_1_2, x_2_2, x_3_2, x_4_2, x_5_2, x_6_2, x_7_2, x_8_2, x_9_2]
[x_0_3, x_1_3, x_2_3, x_3_3, x_4_3, x_5_3, x_6_3, x_7_3, x_8_3, x_9_3]
[[None, p_0_1, p_0_2, p_0_3, p_0_4, p_0_5, p_0_6, p_0_7, p_0_8, p_0_9], [None, None, p_1_2, p_1_3, p_1_4, p_1_5, p_1_6, p_1_7, p_1_8, p_1_9], [None, None, None, p_2_3, p_2_4, p_2_5, p_2_6, p_2_7, p_2_8, p_2_9], [None, None, None, None, p_3_4, p_3_5, p_3_6, p_3_7, p_3_8, p_3_9], [None, None, None, None, None, p_4_5, p_4_6, p_4_7, p_4_8, p_4_9], [None, None, None, None, None, None, p_5_6, p_5_7, p_5_8, p_5_9], [None, None, None, None, None, None, None, p_6_7, p_6_8, p_6_9], [None, None, None, None, None, None, None, None, p_7_8, p_7_9], [None, None, None, None, None, None, None, None, None, p_8_9], [None, None, None, None, None, None, None, None, None, None]]


In [25]:
# Creamos el solver

solver = Solver()

# Agregamos las formulas al solver

# A cada alumno se le asigna una sala

for i in range(10):
    solver.add((sala1[i] | sala2[i] | sala3[i]))

# Cada alumno no puede tener mas de una sala asignada
for i in range(10):
    solver.add(
        (~sala1[i] | ~sala2[i]) &
        (~sala1[i] | ~sala3[i]) &
        (~sala2[i] | ~sala3[i])
    )

# Las parejas en la lista no pueden ir a la misma sala
for i in range(10):
    for j in range(10):
        if i < j:
            solver.add(
                Implies(parejas[i][j], (~sala1[i] | ~sala1[j])) &
                Implies(parejas[i][j], (~sala2[i] | ~sala2[j])) &
                Implies(parejas[i][j], (~sala3[i] | ~sala3[j]))
            )

# La lista de parejas que no pueden estar en la misma sala
solver.add(
    parejas[0][1] &
    parejas[0][5] &
    parejas[0][6] &
    parejas[0][7] &
    parejas[1][3] &
    parejas[1][4] &
    parejas[1][6] &
    parejas[2][5] &
    parejas[2][7] &
    parejas[2][8] &
    parejas[2][9] &
    parejas[3][6] &
    parejas[4][5] &
    parejas[4][6] &
    parejas[5][8] &
    parejas[8][9]
  )

In [26]:
# Ejecutamos el solver y verificamos satisfacibilidad
if solver.check() == sat:
    solucion = solver.model()
    verdaderas = [v for v in solucion if solucion[v] == True and str(v)[0] != "p"]
    print(f"Asignación de salas: {verdaderas}")
else:
    print("No hay una asignacion de salas válida")


Asignación de salas: [x_6_1, x_7_1, x_5_1, x_3_2, x_4_2, x_8_2, x_9_1, x_0_2, x_2_3, x_1_3]
